# Chap03 - 텐서플로의 기본 이해하기

텐서플로의 핵심 구축 및 동작원리를 이해하고, 그래프를 만들고 관리하는 방법과 상수, 플레이스홀더, 변수 등 텐서플로의 '구성 요소'에 대해 알아보자.

## 3.1 연산 그래프

### 3.1.1 연산 그래프란?

그래프는 아래의 그림과 같이 노드(node)나 꼭지점(vertex)로 연결 되어 있는 개체(entity)의 집합을 부르는 용어다. 노드들은 변(edge)을 통해 서로 연결되어 있다. 

![](./images/graph.png)

데이터 흐름 그래프에서(DataFlow Grapy)의 변(edge) 어떤 노드에서 다른 노드로 흘러가는(flow) 데이터의 방향을 정한다.

텐서플로에서 그래프의 각 **노드는 하나의 연산을 나타내며, 입력값을 받아 다른 노드로 전달할 결과값을 출력**한다.

### 3.1.2 연산 그래프의 장점

텐서플로는 그래프의 연결 상태를 기반으로 연산을 최적화한다. 각 그래프에는 노드 간에 의존관계(dependency)가 존재한다.. 예를 들어, 아래의 그림 'A'에서 노드 `e`는 노드 `c`에 **직접의존**(direct dependeny)하고 있고, 노드 `a`에는 **간접의존**(indirect dependency) 한다.

![](./images/graph02.png)

위의 그림에서 노드`e`를 계산하기 위해서는 노드 `c, b, a`만 계산 해주면 된다. 따라서, **의존관계를 이용해 연산량이 최소화**할 수 있다. 이처럼 그래프를 통해 각 노드의 모든 의존관계를 파악할 수 있다.

## 3.2 그래프, 세션, 페치

### 3.2.1 그래프 만들기

`import tensorflow as tf`를 통해 텐서플로를 import 하면 그 시점에 비어 있는 기본 그래프가 만들어지며, 우리가 만드는 모든 노드들은 이 기본 그래프에 자동으로 연결된다.

In [1]:
import tensorflow as tf

다음과 같이 간단한 6개의 노드를 만들어 보자. 먼저 `a, b, c` 노드에 `5, 2, 3`을 대입한다.

In [2]:
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

다음 `d, e, f` 노드에는 `a, b, c`노드를 이용하여 간단한 연산을 수행한다.

In [3]:
d = tf.multiply(a, b)  # a * b
e = tf.add(c, b)  # c + b
f = tf.subtract(d, e)  # d - e

위에서 정의한 노드 및 연산을 그래프로 그려보면 아래와 같다.

<img src="./images/graph03.png" width="60%" height="60%" />

텐서플로에서는 위의 코드처럼 곱셈, 덧셈, 뺄셈을 텐서플로의 `tf.<operator>`를 사용하여 나타낼 수 있을 뿐만아니라 축약 연산자 즉, `*, +, -` 등을 사용할 수 있다.

| TensorFlow 연산      | 축약 연산자 | 설명                                                       |
| -------------------- | ----------- | ---------------------------------------------------------- |
| `tf.add()`           | `a + b`     | a와 b를 더함                                               |
| `tf.multiply()`      | `a * b`     | a와 b를 곱함                                               |
| `tf.subtract()`      | `a - b`     | a에서 b를 뺌                                               |
| `tf.divide()`        | `a / b`     | a를 b로 나눔                                               |
| `tf.pow()`           | `a ** b`    | $a^b$ 를 계산                                              |
| `tf.mod()`           | `a % b`     | a를 b로 나눈 나머지를 구함                                 |
| `tf.logical_and()`   | `a & b`     | a와 b의 논리곱을 구함. `dtype`은 반드시 `tf.bool`이어야 함 |
| `tf.greater()`       | `a > b`     | $a > b$ 의 True/False 값을 반환                            |
| `tf.greater_equal()` | `a >= b`    | $a \ge b$ 의 True/False 값을 반환                          |
| `tf.less_equal()`    | `a <= b`    | $ a \le b$ 의 True/False 값을 반환                         |
| `tf.less()`          | `a < b`     | $a < b$ 의 True/False 값을 반환                            |
| `tf.negative()`      | `-a`        | a의 반대 부호 값을 반환                                    |
| `tf.logical_not()`   | `~a`        | a의 반대의 참거짓을 반환. `tf.bool` 텐서만 적용 가능       |
| `tf.abs()`           | `abs(a)`    | a의 각 원소의 절대값을 반환                              |
| `tf.logical_or()`    | `a I b`     | a와 b의 논리합을 구함. `dtype`은 반드시 `tf.bool`이어야 함 |


### 3.2.2 세션을 만들고 실행하기

3.2.1에서 정의한 노드 및 연산 그래프를 실행하려면 아래의 코드 처럼 **세션(Session)**을 만들고 실행하면 된다.

In [4]:
import tensorflow as tf

# 노드 및 연산 그래프 정의
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

d = tf.multiply(a, b)  # a * b
e = tf.add(c, b)  # c + b
f = tf.subtract(d, e)  # d - e

# 세션을 만들고 연산그래프 실행
sess = tf.Session()
outs = sess.run(f)
sess.close()
print("outs = {}".format(outs))

outs = 5


먼저, `tf.Session()`에서 그래프를 시작한다. `Session`객체는 파이썬 객체와 데이터, 객체의 메모리가 할당되어 있는 실행 환경 사이를 연결하며, 중간 결과를 저장하고 최종 결과를 작업 환경으로 보내준다. 위의 코드에서는 `Session` 객체를 `sess = tf.Session()` 에 정의했다. 

연산 그래프를 실행하려면 `Session`객체의 `run()` 메소드를 사용해야한다. 위의 코드에서 `sess.run(f)`는 아래의 그림처럼 출력이 나와야 하는 `f`노드에서 시작해서 역방향으로 의존관계에 따라 노드의 연산을 수행한다. 

<img src="./images/graph04.png" width="60%" height="60%" />

연산 수행이 완료되면 `sess.close()`를 통해 사용한 메모리를 해제하는 것이 좋다.

### 3.2.3 그래프의 생성과 관리